# HACK DAY SESSION 20

***Transient/object detection simulation: input survey sky coverage/cadence/depth***

***and volumetric object occurrence transient rate/brightness range,*** 

***output rate + number of transients/objects over survey period***

Ananay S, Kovi R, Kaila N

In [1]:
import numpy as np 
import matplotlib.pyplot as plt

from dataclasses import dataclass
from astropy import units as u

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [26]:
@dataclass
class AstroObject:
    """Class containing properties of astronomical objects"""

    obj_types = {0:'TZO',
                 1:'Hot Jupiter',
                 2:'M Dwarf',
                 3:"Star",
                 4:"Supernova"
    }


    st: int
    l_avg: u.Quantity #erg/s
    l_min: u.Quantity #erg/s
    l_max: u.Quantity #erg/s
    vr: u.Quantity #/pc^3

    def __post_init__(self):
        self.object_type = self.obj_types[self.st]
        self.luminosity_avg = self.l_avg.to(u.erg/u.second)
        self.luminosity_min = self.l_min.to(u.erg/u.second)
        self.luminosity_max = self.l_max.to(u.erg/u.second)
        self.volumetric_rate = self.vr.to(1./(u.Mpc**3 * u.year))

    def print_params(self):
        print('Type:',self.object_type)
        print('Volumetric rate:',self.volumetric_rate)
        print('Luminosity peak average:',self.luminosity_avg)
        print('Luminosity peak min/max:',self.luminosity_min,'/',self.luminosity_max)



@dataclass
class Survey:
    """Class containing the basic properties of an astronomical survey"""
    
    spws = {0: 'Radio',
            1: 'Infrared',
            2: 'Visible',
            3: 'Ultraviolet'
    }

    name: str
    spw: int #wavelength range from dictionary
    field_of_view: u.Quantity #normally in square degrees
    noise_thresh: u.Quantity #normally in mJy
    avg_cadence: u.Quantity #normally in days
    total_coverage: u.Quantity #normally in square degrees
    obs_time: u.Quantity #normally in mins
    total_obs_time: u.Quantity #this is how many times (mins) that the object has been observed

    def __post_init__(self):
            self.spectral_window = self.spws[self.spw]
    
    def effective_rate(self, volumetric_rate: u.Quantity):
        vol_fraction = self.total_coverage/(41253*u.deg**2)
        eff_rate = volumetric_rate*vol_fraction
        return eff_rate
    
    # to reduce rms, stacking of epochs
    # fns based on no. obs -> takes cadence from one survey, sensitivty, and calculating what the sensitivity is going to look like now
    
    def predict_next_observation(self, astro_object: AstroObject, num_observations: int = 10):
        """
        Predicts the next sensitivity measurement and cadence value based on stacked observations.

        Args:
        - astro_object: An instance of the AstroObject class containing object properties
        - num_observations: Number of observations to stack for prediction

        Returns:
        - Tuple of predicted sensitivity measurement (u.Quantity) and cadence value (u.Quantity)
        """
        # Generate mock data for sensitivity measurements and cadence values
        sensitivity_data = np.random.uniform(1, 10, num_observations) * self.noise_thresh
        cadence_data = np.random.uniform(0.5, 1.5, num_observations) * self.avg_cadence

        # Use AstroObject properties to influence prediction
        luminosity_avg = astro_object.luminosity_avg.to(u.erg / u.s).value
        volumetric_rate = astro_object.volumetric_rate.to(1/(u.Mpc**3 * u.year)).value

        # Example calculation: reduce noise threshold and increase total coverage
        reduced_noise_thresh = self.noise_thresh / np.sqrt(num_observations)
        increased_total_coverage = self.total_coverage * num_observations
        # Calculate number of detections
        num_detections = int(self.total_obs_time / self.obs_time * num_observations)

        return reduced_noise_thresh, increased_total_coverage, num_detections


In [27]:
# Example
vast_survey = Survey("VAST", spw=0, field_of_view=30, noise_thresh=0.25, avg_cadence=30, total_coverage=1e4, obs_time=15, total_obs_time=900)
vast_survey

Survey(name='VAST', spw=0, field_of_view=30, noise_thresh=0.25, avg_cadence=30, total_coverage=10000.0, obs_time=15, total_obs_time=900)

In [28]:
# Example
h = 0.7
vr_sn = (1.06e-4)*((h/0.7)**3)/(u.Mpc**3 * u.year)

L_peak = (10**25.5)*u.erg/(u.second)
L_low = (10**23.5)*u.erg/(u.second)
L_high = (10**29)*u.erg/(u.second)

sn = AstroObject(4,L_peak,L_low,L_high,vr_sn)

sn.print_params()


Type: Supernova
Volumetric rate: 0.000106 1 / (Mpc3 yr)
Luminosity peak average: 3.1622776601683795e+25 erg / s
Luminosity peak min/max: 3.162277660168379e+23 erg / s / 1e+29 erg / s


In [32]:
# Test Case
# Test the predict_next_observation method of Survey
predicted_sensitivity, predicted_coverage, num_detections = vast_survey.predict_next_observation(sn,num_observations=60)

# Print the predicted values
print("Predicted Sensitivity:", predicted_sensitivity, "mJy")
print("Predicted Coverage:", predicted_coverage, "deg squared")
print("Number of Detections per year:", num_detections)

Predicted Sensitivity: 0.03227486121839514 mJy
Predicted Coverage: 600000.0 deg squared
Number of Detections: 3600


In [30]:
# TZO Parameters

vr_tzo = ((4e-4)*2.26e-7/(u.year*u.Mpc**3))
print(vr_tzo)
tzo = AstroObject(0,(7.564540e+04)*u.Lsun, (9.990184e-01)*u.Lsun, (2.924590e+06)*u.Lsun, vr_tzo)
tzo.print_params()

9.04e-11 1 / (Mpc3 yr)
Type: TZO
Volumetric rate: 9.04e-11 1 / (Mpc3 yr)
Luminosity peak average: 2.8957059119999994e+38 erg / s
Luminosity peak min/max: 3.8242424352e+33 erg / s / 1.119533052e+40 erg / s


In [ ]:
# -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Kovi's code

################ Survey Object Class #####
@dataclass
class Survey:
    """Class containing the basic properties of an astronomical survey"""
    
    spws = {0: 'Radio',
            1: 'Infrared',
            2: 'Visible',
            3: 'Ultraviolet'
    }

    name: str
    spw: int #wavelength range from dictionary
    field_of_view: u.Quantity #normally in square degrees
    noise_thresh: u.Quantity #normally in mJy
    avg_cadence: u.Quantity #normally in days
    total_coverage: u.Quantity #normally in square degrees
    standard_obs_duration: u.Quantity #normally in seconds
    obs_duration: u.Quantity #normally in seconds
    
    def __post_init__(self):
            self.spectral_window = self.spws[self.spw]

    def volume_limit(self, sigma: float, max_luminosity: u.Quantity):
        flux_limit = sigma*self.noise_thresh*np.sqrt(self.standard_obs_duration/self.obs_duration)
        dist_limit = ((max_luminosity/(flux_limit*4*np.pi))**0.5).to(u.Mpc)
        vol_limit = (4/3*np.pi*dist_limit**3)#*u.Mpc**3
        return dist_limit, vol_limit
    
    def effective_rate(self, volumetric_rate: u.Quantity):
        vol_fraction = self.total_coverage/(41253*u.deg**2)
        eff_rate = volumetric_rate*vol_fraction
        return eff_rate
    
#TODO: add print_params to survey class
    
################################################################

########## Astro Object Class ################

@dataclass
class AstroObject:
    """Class containing properties of astronomical objects"""

    obj_types = {0:'TZO',
                 1:'Hot Jupiter',
                 2:'M Dwarf',
                 3:"Star",
                 4:"Supernova"
    }

    st: int
    l_avg: u.Quantity #erg/s
    l_min: u.Quantity #erg/s
    l_max: u.Quantity #erg/s
    vr: u.Quantity #/pc^3

    def __post_init__(self):
        self.object_type = self.obj_types[self.st]
        self.luminosity_avg = self.l_avg.to(u.erg/u.second/u.Hz)
        self.luminosity_min = self.l_min.to(u.erg/u.second/u.Hz)
        self.luminosity_max = self.l_max.to(u.erg/u.second/u.Hz)
        self.volumetric_rate = self.vr.to(1./(u.Mpc**3 * u.year))

    def print_params(self):
        print('Type:',self.object_type)
        print('Volumetric rate:',self.volumetric_rate)
        print('Luminosity peak average:',self.luminosity_avg)
        print('Luminosity peak min/max:',self.luminosity_min,'/',self.luminosity_max)
    
#######################################################################################
            
############## Non-Self Functions ########